In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df_tot = pd.read_csv("final.csv")

In [3]:
# per rispondere alle mie domande mi basta avere gli elements, quindi elimino le rige relative ai components
print("initial df.shape:",df_tot.shape)
df = df_tot.drop(df_tot[df_tot.section == "components"].index)
print("post cut df.shape:",df.shape)

initial df.shape: (214491, 19)
post cut df.shape: (136861, 19)


In [4]:
# Chi sono i giudici più buoni?
# grafico heatmap:
# x: atleti
# y: giudici
# heat: differenza percentuale tra il voto medio dato da quel giudice a
# quell'atleta e quello medio degli altri giudici dato a quell'atleta
# per rispondere mi creo una tabella con la media dei voti che ciascun giudice ha dato
# a ciascun atleta
df.head(1)

,competition,program,performance_id,athlete_name,athlete_nation,rank,total_segment_score,total_element_score,total_component_score,total_deductions,aspect_id,aspect_desc,element_order,element_base_value,section,aspect_final_score,judge_name,judge_nation,judge_score
2,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,38bda018f5,3Lo,4.0,5.61,elements,7.21,Agita ABELE,LAT,3.0


In [5]:
# elimino le colonne che non mi servono più
df.drop('total_segment_score', axis=1, inplace=True)
df.drop('total_component_score', axis=1, inplace=True)
df.drop('total_element_score', axis=1, inplace=True)
df.drop('section', axis=1, inplace=True)
df_q2 = df.drop('competition', axis=1)
df_q2.drop('program', axis=1, inplace=True)
df_q2.drop('performance_id', axis=1, inplace=True)
df_q2.drop('rank', axis=1, inplace=True)
df_q2.drop('total_deductions', axis=1, inplace=True)
df_q2.drop('aspect_id', axis=1, inplace=True)
df_q2.drop('aspect_desc', axis=1, inplace=True)
df_q2.drop('element_order', axis=1, inplace=True)
df_q2.drop('element_base_value', axis=1, inplace=True)
df_q2.drop('aspect_final_score', axis=1, inplace=True)
df_q2.drop('judge_nation', axis=1, inplace=True)
df_q2.drop('athlete_nation', axis=1, inplace=True)
df_q2.head(2)

,athlete_name,judge_name,judge_score
2,Evgenia MEDVEDEVA,Agita ABELE,3.0
3,Evgenia MEDVEDEVA,Agita ABELE,3.0


In [6]:
# creo la tabella atleti_voti, che alla fine sarà formata nel seguente modo:
# - athlete_name
# - judge_name
# - mean_judge
# - mean_overall
# - diff_perc
voti = df_q2.groupby(['athlete_name','judge_name'])['judge_score'].agg(['mean']).reset_index()
voti.rename(columns={'mean': 'mean_judge'}, inplace=True)

In [7]:
voti.head(5)

,athlete_name,judge_name,mean_judge
0,Adam RIPPON,Agita ABELE,1.20
1,Adam RIPPON,Alexei BELETSKI,1.15
2,Adam RIPPON,Anna KANTOR,0.80
3,Adam RIPPON,Anthony LEROY,1.35
4,Adam RIPPON,Ayumi KOZUKA,0.15


In [8]:
tmp = voti.groupby(['athlete_name'])['mean_judge'].agg(['mean']).reset_index()
tmp.rename(columns={'mean': 'mean_overall'}, inplace=True)

In [9]:
tmp.head()

,athlete_name,mean_overall
0,Adam RIPPON,1.171591
1,Adel TANKOVA / Ronald ZILBERBERG,-0.577778
2,Adele MORRISON / Demid ROKACHEV,-0.611111
3,Aimee BUCHANAN,-0.412698
4,Alaine CHARTRAND,-0.206160


In [10]:
print('voti.shape after: ',voti.shape)
print('tmp.shape after: ',tmp.shape)
voti = voti.merge(tmp,
                  how='left',
                  on='athlete_name')

print('voti.shape after: ',voti.shape)

voti.shape after:  (7595, 3)
tmp.shape after:  (319, 2)
voti.shape after:  (7595, 4)


In [11]:
voti.head(5)

,athlete_name,judge_name,mean_judge,mean_overall
0,Adam RIPPON,Agita ABELE,1.20,1.171591
1,Adam RIPPON,Alexei BELETSKI,1.15,1.171591
2,Adam RIPPON,Anna KANTOR,0.80,1.171591
3,Adam RIPPON,Anthony LEROY,1.35,1.171591
4,Adam RIPPON,Ayumi KOZUKA,0.15,1.171591


In [12]:
voti["diff_perc"] = np.nan

In [13]:
for ind in voti.index:
    voti.at[ind,'diff_perc'] = (abs(voti['mean_overall'][ind] - voti['mean_judge'][ind]) / voti['mean_judge'][ind]) * 100.0

<ipython-input-13-463247f9b906>:2: RuntimeWarning: divide by zero encountered in double_scalars
  voti.at[ind,'diff_perc'] = (abs(voti['mean_overall'][ind] - voti['mean_judge'][ind]) / voti['mean_judge'][ind]) * 100.0
<ipython-input-13-463247f9b906>:2: RuntimeWarning: invalid value encountered in double_scalars
  voti.at[ind,'diff_perc'] = (abs(voti['mean_overall'][ind] - voti['mean_judge'][ind]) / voti['mean_judge'][ind]) * 100.0


In [14]:
voti.head()

,athlete_name,judge_name,mean_judge,mean_overall,diff_perc
0,Adam RIPPON,Agita ABELE,1.20,1.171591,2.367424
1,Adam RIPPON,Alexei BELETSKI,1.15,1.171591,1.877470
2,Adam RIPPON,Anna KANTOR,0.80,1.171591,46.448864
3,Adam RIPPON,Anthony LEROY,1.35,1.171591,13.215488
4,Adam RIPPON,Ayumi KOZUKA,0.15,1.171591,681.060606


In [15]:
# elimino le colonne che non mi servono per la heatmap
voti.drop('mean_judge', axis=1, inplace=True)
voti.drop('mean_overall', axis=1, inplace=True)

In [16]:
voti.head()

,athlete_name,judge_name,diff_perc
0,Adam RIPPON,Agita ABELE,2.367424
1,Adam RIPPON,Alexei BELETSKI,1.877470
2,Adam RIPPON,Anna KANTOR,46.448864
3,Adam RIPPON,Anthony LEROY,13.215488
4,Adam RIPPON,Ayumi KOZUKA,681.060606


In [17]:
pivoted = voti.pivot(index="athlete_name", columns="judge_name", values="diff_perc")
pivoted.head()

judge_name,Adriana DOMANSKA,Agita ABELE,Agnieszka SWIDERSKA,Aigul KUANISHEVA,Akiko KOBAYASHI,Akira YOSHIZAWA,Akos PETHES,Albert ZAYDMAN,Alexandre GOROJDANOV,Alexei BELETSKI,...,Wei SHI,Weiguang CHEN,Wendy ENZMANN,Xixia LIU,Yoko KUNO,Yu WANG,Yuko OGAWA,Yuri GUSKOV,Zanna KULIK,Zuzana PLESNIKOVA
athlete_name,,,,,,,,,,,,,,,,,,,,,
Adam RIPPON,NaN,2.367424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.87747,...,NaN,4.141414,16.314935,NaN,26.775568,NaN,NaN,14.793388,NaN,NaN
Adel TANKOVA / Ronald ZILBERBERG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adele MORRISON / Demid ROKACHEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aimee BUCHANAN,NaN,-3.703704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-44.444444,NaN
Alaine CHARTRAND,NaN,NaN,NaN,NaN,NaN,295.852466,NaN,NaN,NaN,NaN,...,NaN,-44.042153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
pivoted.shape

(319, 214)

In [19]:
fig = px.imshow(pivoted,zmin=-100,zmax=100)
fig.show()